In [1]:
import nltk
import glob
import csv
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
from nltk.corpus import stopwords
from string import punctuation
stopwords = stopwords.words('english')
from nltk import word_tokenize
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-04-22 01:15:36,102 : WARNING : Slow version of gensim.models.doc2vec is being used
2017-04-22 01:15:36,133 : INFO : 'pattern' package not found; tag filters are not available for English


In [2]:
files = pd.read_csv('..\csv\\filenameinfo.csv')

In [11]:
x = files.sample(100)
x['filename'].head()

971            ..\Data\1785081.female.24.indUnk.Pisces.xml
11286    ..\Data\3812790.female.33.Religion.Sagittarius...
6112            ..\Data\3466588.female.16.indUnk.Virgo.xml
11593       ..\Data\3832680.male.15.indUnk.Sagittarius.xml
18363          ..\Data\4287659.female.15.Student.Aries.xml
Name: filename, dtype: object

In [12]:
from bs4 import BeautifulSoup
docs = []
for name in x['filename'] :
    with open(name, 'r') as f :
        try :
            soup = BeautifulSoup(f, "xml", from_encoding='utf8')
        except UnicodeDecodeError :
            continue
    posts = [cleanText(post.get_text()) for post in soup.find_all('post')]
    docs.extend(posts)


In [7]:
def cleanText(post) :
    x = exclude = set(punctuation)
    s = ''.join(ch for ch in post if ch not in exclude)
    words = [word.lower() for word in word_tokenize(s) if word.lower() not in stopwords 
                                                           and word not in ['urllink']]
    return words

In [6]:
posts = soup.find_all('post')

docs = []

for post in posts :
    doc = cleanText(post.get_text().strip())
    docs.append(doc)

print(docs)

[['online', 'looking', 'pokemon', 'episode', 'guides', 'found', 'pokemon', 'housou', 'translated', 'means', 'pokemon', 'broadcast', 'english', 'separate', 'show', 'actual', 'pokemon', 'tv', 'series', 'shows', 'goes', 'places', 'ash', 'away', 'hoenn', 'show', 'currently', 'airs', 'japan', 'appears', 'good', 'cartoon', 'series', 'thing', 'air', 'usa', 'uk', 'english', 'wait', 'see', 'begin', 'airing', 'episodes', 'tey', 'stick', 'current', 'pokemon', 'advance', 'first', 'aired', 'december', '2002', 'japanese', 'tv', 'tuesdays', '700', 'pm'], ['may', 'may', 'know', 'trinicom', 'local', 'trinidad', 'tobago', 'website', 'problem', 'used', 'good', 'site', 'one', 'point', 'recently', '2', 'months', 'ago', 'seems', 'going', 'drain', 'comes', 'services', 'sms', 'chat', 'services', 'currently', 'dont', 'antispyware', 'software', 'installed', 'trap', 'spyware', 'programs', 'using', 'sms', 'service', 'used', 'using', 'time', 'tried', 'send', 'message', 'would', 'stop', 'go', 'next', 'page', 'full'

In [14]:
from collections import defaultdict
frequency = defaultdict(int)
for text in docs:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in docs]

print(texts)

[['told', 'today', 'try', 'beg', 'beg', 'denied', 'hurts', 'much', 'wouldnt', 'want', 'put', 'hope', 'fled', 'lost', 'life', 'wanted', 'truly', 'gone', 'forever', 'quit', 'hoping', 'quit', 'memories', 'coming', 'quit', 'pain', 'filling', 'every', 'moment', 'even', 'dying', 'wouldnt', 'enough', 'nothing', 'would', 'enough', 'shes', 'without', 'hope', 'dont', 'want', 'live', 'anymore', 'dont', 'want', 'anymore', 'im', 'everything'], ['wonder', 'long', 'wake', 'without', 'thinking', 'without', 'wondering', 'shes', 'thinking', 'long', 'longer', 'wake', 'memories', 'us', 'together', 'like', 'small', 'mirror', 'reflecting', 'good', 'never', 'like', 'little', 'shards', 'glass', 'every', 'little', 'one', 'brings', 'different', 'new', 'pain', 'embedded', 'skin', 'wake', 'without', 'heavy', 'weight', 'regret', 'shoulders', 'without', 'wish', 'cry', 'without', 'wish', 'disappear', 'stop', 'showing', 'dreams', 'haunting', 'even', 'sleep', 'stop', 'thinking', 'dreaming', 'trying', 'find', 'ways', '

In [15]:
dictionary = corpora.Dictionary(docs)
dictionary.save('test.dict')
print(dictionary)

2017-04-22 01:28:35,960 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-04-22 01:28:36,254 : INFO : built Dictionary(25693 unique tokens: ['told', 'today', 'try', 'beg', 'denied']...) from 1540 documents (total 162711 corpus positions)
2017-04-22 01:28:36,254 : INFO : saving Dictionary object under test.dict, separately None
2017-04-22 01:28:36,270 : INFO : saved test.dict


Dictionary(25693 unique tokens: ['told', 'today', 'try', 'beg', 'denied']...)


In [9]:
new_doc = "Life of the party"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)

[(274, 1)]


In [16]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('test.mm', corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2017-04-22 01:28:48,662 : INFO : storing corpus in Matrix Market format to test.mm
2017-04-22 01:28:48,662 : INFO : saving sparse matrix to test.mm
2017-04-22 01:28:48,662 : INFO : PROGRESS: saving document #0
2017-04-22 01:28:48,834 : INFO : PROGRESS: saving document #1000
2017-04-22 01:28:48,912 : INFO : saved 1540x25678 matrix, density=0.277% (109383/39544120)
2017-04-22 01:28:48,928 : INFO : saving MmCorpus index to test.mm.index


[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2), (8, 3), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 2), (36, 1), (37, 1)]
[(0, 1), (6, 2), (9, 1), (13, 1), (17, 1), (20, 2), (22, 4), (24, 1), (26, 2), (30, 1), (31, 1), (32, 6), (33, 1), (34, 2), (35, 1), (36, 2), (38, 1), (39, 2), (40, 4), (41, 3), (42, 1), (43, 1), (45, 1), (46, 1), (47, 3), (48, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 2), (55, 2), (56, 2), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 5), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98

In [17]:
tfidf = models.TfidfModel(corpus)
print(tfidf)

2017-04-22 01:29:01,972 : INFO : collecting document frequencies
2017-04-22 01:29:01,987 : INFO : PROGRESS: processing document #0
2017-04-22 01:29:02,018 : INFO : calculating IDF weights for 1540 documents and 25677 features (109383 matrix non-zeros)


TfidfModel(num_docs=1540, num_nnz=109383)


In [18]:
corpus_tfidf = tfidf[corpus]

In [19]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf]
lsi.print_topics(10)

2017-04-22 01:29:20,206 : INFO : using serial LSI version on this node
2017-04-22 01:29:20,206 : INFO : updating model with new documents
2017-04-22 01:29:20,447 : INFO : preparing a new chunk of documents
2017-04-22 01:29:20,478 : INFO : using 100 extra samples and 2 power iterations
2017-04-22 01:29:20,478 : INFO : 1st phase: constructing (25693, 110) action matrix
2017-04-22 01:29:20,541 : INFO : orthonormalizing (25693, 110) action matrix
2017-04-22 01:29:21,345 : INFO : 2nd phase: running dense svd on (110, 1540) matrix
2017-04-22 01:29:21,423 : INFO : computing the final decomposition
2017-04-22 01:29:21,423 : INFO : keeping 10 factors (discarding 68.242% of energy spectrum)
2017-04-22 01:29:21,439 : INFO : processed documents up to #1540
2017-04-22 01:29:21,455 : INFO : topic #0(5.924): 0.298*"urllink" + 0.165*"im" + 0.125*"dont" + 0.119*"like" + 0.110*"really" + 0.106*"know" + 0.100*"get" + 0.098*"well" + 0.096*"got" + 0.096*"think"
2017-04-22 01:29:21,470 : INFO : topic #1(5.1

[(0,
  '0.298*"urllink" + 0.165*"im" + 0.125*"dont" + 0.119*"like" + 0.110*"really" + 0.106*"know" + 0.100*"get" + 0.098*"well" + 0.096*"got" + 0.096*"think"'),
 (1,
  '-0.941*"urllink" + 0.060*"im" + 0.047*"dont" + -0.047*"terrance" + 0.039*"like" + 0.038*"know" + 0.038*"really" + 0.036*"well" + 0.036*"time" + 0.035*"get"'),
 (2,
  '0.633*"ja" + 0.272*"ettã¤" + 0.227*"ei" + 0.193*"se" + 0.175*"kun" + 0.160*"mutta" + 0.152*"sitten" + 0.136*"niin" + 0.119*"ihan" + 0.119*"en"'),
 (3,
  '0.194*"went" + 0.165*"lol" + 0.159*"band" + 0.154*"na" + 0.134*"u" + 0.130*"got" + 0.125*"lolx" + -0.120*"poem" + 0.118*"gon" + 0.116*"haha"'),
 (4,
  '-0.378*"terrance" + -0.265*"krys" + -0.193*"poem" + 0.188*"dont" + -0.178*"new" + 0.159*"im" + -0.145*"got" + -0.112*"went" + 0.111*"feel" + 0.111*"love"'),
 (5,
  '0.607*"terrance" + 0.450*"krys" + -0.181*"poem" + 0.155*"trina" + -0.147*"new" + -0.133*"got" + 0.098*"dont" + -0.092*"poetry" + 0.083*"feel" + 0.079*"sling"'),
 (6,
  '-0.285*"u" + 0.216*"im" 

In [20]:
f = open('mycorpora.txt', 'w+')
dictionary = corpora.Dictionary.load('test.dict')
x = files.sample(10)
for name in x['filename'] :
    with open(name, 'r', errors='ignore') as f :
        soup = BeautifulSoup(f, "xml")
    posts = soup.find_all('post')

    docs = []

    for post in posts :
        doc = cleanText(post.get_text().strip())
        docs.append(doc)
    frequency = defaultdict(int)
    for text in docs:
        for token in text:
            frequency[token] += 1

    texts = [[token for token in text if frequency[token] > 5] for text in docs]
    
    for text in texts :
        f.write(''.join(word for word in text) + '\n')
    dictionary.add_documents(texts)

f.close()
dictionary.save('test.dict')

2017-04-18 01:29:23,450 : INFO : loading Dictionary object from test.dict
2017-04-18 01:29:23,450 : INFO : loaded test.dict


ValueError: I/O operation on closed file.